In [50]:
import numpy as np
import pandas as pd
import json, requests
from tmdbv3api import TMDb
tmdb = TMDb()
tmdb.api_key = 'e3dce8cf7d87efd38c89a43d9fe6d597'

In [51]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"
df1 = pd.read_html(link, header=0)[3]
df2 = pd.read_html(link, header=0)[4]
df3 = pd.read_html(link, header=0)[5]
df4 = pd.read_html(link, header=0)[6]

In [52]:
df = df1.append(df2.append(df3.append(df4, ignore_index=True), ignore_index=True), ignore_index=True)

In [53]:
df.isnull().sum()

Opening                0
Opening.1              0
Title                  0
Production company     0
Cast and crew          0
Ref.                  27
dtype: int64

In [54]:
df.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2]
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3]
2,JANUARY,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4]
3,JANUARY,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN
4,JANUARY,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5]


In [55]:
from tmdbv3api import Movie
tmdb_movie = Movie()

def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    try:
        movie_id = result[0].id
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id, tmdb.api_key))
        json_data = response.json()
        if json_data['genres']:
            str_ = " "
            for i in range(0, len(json_data['genres'])):
                genres.append(json_data['genres'][i]['name'])
            return str_.join(genres)
        else:
            return np.NaN
    except IndexError:
        return np.NaN
        

In [56]:
df['genres'] = df['Title'].map(lambda x: get_genre(str(x)))

In [57]:
df.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.,genres
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2],Horror Mystery Thriller
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3],Action Horror Science Fiction Thriller
2,JANUARY,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4],Comedy
3,JANUARY,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN,Drama
4,JANUARY,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5],Drama Thriller Crime


In [58]:
df_2020 = df[['Title','Cast and crew', 'genres']]

In [59]:
df_2020.isna().sum()

Title            0
Cast and crew    0
genres           1
dtype: int64

In [60]:
df_2020[df_2020['genres'].isnull()]

,Title,Cast and crew,genres
159,Killian & the Comeback Kids,Taylor A. Purdee (director/screenplay); Taylor...,NaN


In [61]:
df_2020.fillna('', inplace=True)

/home/mehant/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


### Extrating Director

In [62]:
def get_director(x):
    if " (director)" in x:
        return x.partition(' (director)')[0]
    elif " (directors)" in x:
        return x.partition(' (directors)')[0]
    else:
        return x.partition(' (director/screenplay)')[0]

In [63]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(x))

<ipython-input-63-0d29c98c2cf5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(x))


In [64]:
df_2020.head()

,Title,Cast and crew,genres,director_name
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller,Nicolas Pesce
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller,William Eubank
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy,Miguel Arteta
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,Drama,Jon Avnet
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Drama Thriller Crime,Anthony Jerjen


### Extrating actor1, actor2, actor3

In [65]:
def get_actor1(x):
    return ((x.partition("screenplay); ")[-1]).split(", ")[0]).strip()

In [66]:
df_2020['actor1'] = df_2020['Cast and crew'].map(lambda x: get_actor1(x))

<ipython-input-66-2941e4a42157>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['actor1'] = df_2020['Cast and crew'].map(lambda x: get_actor1(x))


In [67]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [68]:
df_2020['actor2'] = df_2020['Cast and crew'].map(lambda x: get_actor2(x))

<ipython-input-68-42b68ed97c2a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['actor2'] = df_2020['Cast and crew'].map(lambda x: get_actor2(x))


In [69]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [70]:
df_2020['actor3'] = df_2020['Cast and crew'].map(lambda x: get_actor3(x))

In [71]:
df_2020.drop(columns={'Cast and crew'}, inplace=True)

In [72]:
df_2020 = df_2020.rename(columns={'Title':'title','director_name':'director'})

In [73]:
df_2020['title'] = df_2020['title'].str.lower()

### Delling with NaN values

In [74]:
df_2020.isna().sum()

title        0
genres       0
director     0
actor1       0
actor2       3
actor3      26
dtype: int64

In [78]:
df_2020.fillna('', inplace=True)

In [79]:
df_2020.isna().sum()

title       0
genres      0
director    0
actor1      0
actor2      0
actor3      0
dtype: int64

In [80]:
df_2020['comb'] = df_2020.actor1 + ' ' + df_2020.actor2 + ' ' + df_2020.actor3 + ' ' + df_2020.director + ' ' + df_2020.genres

In [82]:
df_2020

,title,genres,director,actor1,actor2,actor3,comb
0,the grudge,Horror Mystery Thriller,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Andrea Riseborough Demián Bichir John Cho Nico...
1,underwater,Action Horror Science Fiction Thriller,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Kristen Stewart Vincent Cassel Jessica Henwick...
2,like a boss,Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,three christs,Drama,Jon Avnet,Eric Nazarian (screenplay); Richard Gere,Peter Dinklage,Walton Goggins,Eric Nazarian (screenplay); Richard Gere Peter...
4,inherit the viper,Drama Thriller Crime,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Josh Hartnett Margarita Levieva Chandler Riggs...
...,...,...,...,...,...,...,...
264,we can be heroes,Action Fantasy Family Drama Comedy,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...
265,news of the world,Action Adventure Drama Western,Paul Greengrass,Luke Davies (screenplay); Tom Hanks,Helena Zengel,,Luke Davies (screenplay); Tom Hanks Helena Zen...
266,one night in miami,Drama,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...
267,promising young woman,Thriller Crime Drama,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Carey Mulligan Bo Burnham Alison Brie Emerald ...


In [83]:
df_2020.to_csv("wiki_2020.csv", index=False)